In [159]:
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [160]:
tokenizer = get_tokenizer('basic_english')
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

In [182]:
def return_iters():
    from ast import literal_eval as make_tuple
    train_iter=[]
    test_iter=[]
    file = open("PoliticalCompassML/DB",'r')
    # file2 = open("PoliticalCompassML/databaseTop.txt",'r')
    mapping = {
    "Libertarian Left":1,
    "Libertarian Right":2,
    "Authoritarian Left":3,
    "Authoritarian Right":4,
    }
    for line in file:
        opinion, text = make_tuple(line)
        train_iter+=[(mapping[opinion],text)]
        test_iter+=[(mapping[opinion],text)]
    # for line in file2:
    #     opinion, text = make_tuple(line)
    #     train_iter+=[(mapping[opinion],text)]
    #     test_iter+=[(mapping[opinion],text)]
    train_iter=iter(train_iter)
    test_iter=iter(test_iter)
    file.close()
    # file2.close()
    return train_iter,test_iter

In [183]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1
train_iter,test_iter= return_iters()
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])


In [184]:
import torch
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [185]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

In [186]:
train_iter,test_iter= return_iters()

dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [187]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [188]:
train_iter,test_iter= return_iters()

num_class = len(set([label for (label, text) in train_iter]))
# print(train_iter)
# print([label for (label, text) in train_iter])
print(num_class)
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

4


In [189]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 50
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [190]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 40 # epoch
LR = 60  # learning rate
BATCH_SIZE = 128 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter,test_iter= return_iters()

train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |    50/  188 batches | accuracy    0.331
| epoch   1 |   100/  188 batches | accuracy    0.340
| epoch   1 |   150/  188 batches | accuracy    0.352
-----------------------------------------------------------
| end of epoch   1 | time:  1.76s | valid accuracy    0.361 
-----------------------------------------------------------
| epoch   2 |    50/  188 batches | accuracy    0.329
| epoch   2 |   100/  188 batches | accuracy    0.349
| epoch   2 |   150/  188 batches | accuracy    0.346
-----------------------------------------------------------
| end of epoch   2 | time:  1.46s | valid accuracy    0.363 
-----------------------------------------------------------
| epoch   3 |    50/  188 batches | accuracy    0.344
| epoch   3 |   100/  188 batches | accuracy    0.360
| epoch   3 |   150/  188 batches | accuracy    0.364
-----------------------------------------------------------
| end of epoch   3 | time:  1.52s | valid accuracy    0.181 
-------------------------------

KeyboardInterrupt: 

In [90]:
mapping = {
"Libertarian Left":1,
"Libertarian Right":2,
"Authoritarian Left":3,
"Authoritarian Right":4,
}
def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

ex_text_str = "Would you like the biggest socioeconomic reason wages have been held down and now need two incomes instead of one for a household? Nobody likes the answer."
#https://old.reddit.com/r/PoliticalCompassMemes/comments/x774os/conservative_you_say_sounds_fine_to_me/inapp39/
model = model.to("cpu")

print("This is a %s news" %mapping[predict(ex_text_str, text_pipeline)])

KeyError: 1